<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Image annotation UI

Open-source annotation tools for object detection and for image segmentation exist, however for image classification we were not able to find a good program. Hence this notebook provides a simple UI to label images. Each image can be annotated with one or multiple classes, or marked as "Exclude" to indicate that the image should not be used for model trainining or evaluation.

In [1]:
# Ensure edits to libraries are loaded and plotting is shown in the notebook.
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append("../")
from utils_ic.anno_utils import AnnotationWidget

Set parameters: location of the images to annotate, and path where to save the annotations.

In [3]:
IM_DIR = "../data/fridgeObjects/can"
ANNO_PATH = "cvbp_ic_annotation.pkl"

Start the UI. Check the "Allow multi-class labeling" box to allow that images can be annotated as multiple classes. When in doubt what the annotation for an image should be, or for any other reason (e.g. blur or over-exposure), mark an image as "EXCLUDE". All annotations are saved to (and loaded from) a pandas dataframe with path specified in `anno_path`. 

<center>
<img src="https://cvbp.blob.core.windows.net/public/images/document_images/anno_ui2.jpg" style="width: 600px;"/>
<i>Annotation UI example</i>
</center>

In [4]:
w_anno_ui = AnnotationWidget(
    labels       = ["can", "carton", "milk_bottle", "water_bottle"],
    im_dir       = IM_DIR,
    anno_path    = ANNO_PATH,
    im_filenames = None #Set to None to annotate all images in IM_DIR
)

display(w_anno_ui.show())

Loading existing annotation from cvbp_ic_annotation.pkl.


Below is an example how to create a fast.ai ImageList object using the ground truth annotations generated by the AnnotationWidget. Note that fast.ai does not support the exclude flag, hence we remove these images before calling fast.ai's `from_df()` and `label_from_df()` functions.

```python
import pandas as pd
from fastai.vision import ImageList,ImageDataBunch

# Load annotation, discard excluded images, and convert to format fastai expects
annos = pd.read_pickle(ANNO_PATH) 
keys = [key for key in annos if (not annos[key].exclude and len(annos[key].labels)>0)]
df = pd.DataFrame([(anno[0], ",".join(anno[1].labels)) for anno in annos[keys].items()], 
                  columns = ["name", "label"])
display(df)

# Create an ImageList and assign labels 
data = (ImageList.from_df(path=IM_DIR, df = df)
       .split_by_rand_pct(valid_pct=0.5)
       .label_from_df(label_delim=','))
```